**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath("/usr/local/bin")
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using Theano backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)
        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function $\textit{act}$ returns an action $a$ given a state $s$. 
More precisely, 

- if we are in the training process, either it returns the action randomly with probability $\epsilon$ or it returns the action according to the policy $\pi$ with probability $1-\epsilon$. $\textit{Why?}$
>We want to exploit and explore the environment. And, to explore the environment the function takes randomly with probability $\epsilon$ an action and, to exploit the environment it takes the action that it expects to give the largest reward. And it is the policy $\pi$ which learns to return the action which maximizes the reward given a state. 
Therefore, $\epsilon$ is essential for the exploration process.

- if we are in the testing process, the function $act$ just follows the optimal policy learnt during the training process. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=300 # set small when debugging
epochs_test=40 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The $position$ array is used to find out where you can go and it takes into account the 2 cell visibility (this is why the size of the table is 4 larger cells). Indeed, if the cell is equal to $-1$ then the agent is not allowed to be in it and if the cell is equal to $0$ it means that you are allowed to be in it. In addition, the cell of your position is equal to 1. 

The $board$ array gives for each cell the associated reward : $0$ corresponds to no reward, $0.5$ corresponds to cheese and $-1$ corresponds to poison.  

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        self.n_action = 4

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/14.0. Average score (-5.0)
Win/lose count 13.5/17.0. Average score (-4.25)
Win/lose count 8.5/19.0. Average score (-6.333333333333333)
Win/lose count 9.0/13.0. Average score (-5.75)
Win/lose count 11.5/12.0. Average score (-4.7)
Win/lose count 13.0/19.0. Average score (-4.916666666666667)
Win/lose count 10.5/16.0. Average score (-5.0)
Win/lose count 10.0/10.0. Average score (-4.375)
Win/lose count 9.0/16.0. Average score (-4.666666666666667)
Win/lose count 7.0/7.0. Average score (-4.2)
Win/lose count 9.5/11.0. Average score (-3.9545454545454546)
Win/lose count 14.0/14.0. Average score (-3.625)
Win/lose count 9.0/14.0. Average score (-3.730769230769231)
Win/lose count 12.0/17.0. Average score (-3.8214285714285716)
Win/lose count 13.0/16.0. Average score (-3.7666666666666666)
Win/lose count 10.5/13.0. Average score (-3.6875)
Win/lose count 10.0/15.0. Average score (-3.764705882352941)
Win/lose count 7.0/21.0. Average score (-4.333333333333333)
Win/lose count 9.0/11.0. 

In [13]:
HTML(display_videos('random'+str(epochs_test-1)+'.mp4'))

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1) We know that 
$$\begin{align*} Q^\pi(s,a) & = \mathbb{E}_{p^\pi}\left[ \sum_{t=0}^T \gamma^t r(s_t,a_t) | s_0 = s, a_0 = a\right] \\
&= \mathbb{E}_{a_0, a1, s1, \ldots, s_T} \left[ \gamma^0 r(s_0, a_0) |s_0 = s, a_0 = a \right] + \mathbb{E}_{a_0, a_1, s1, \ldots, s_T}\left[ \sum_{t=1}^T \gamma^t r(s_t, a_t) | s_0 =s, a_0 = a \right]\\
& = r(s, a) + \mathbb{E} \left[ \mathbb{E}\left[\sum_{t=1}^T \gamma^t r(s_t, a_t) |s_1, a_1\right] | s_0 =s, a_0 = a \right] \\
& = r(s, a) + \sum_{s', a'} p(s_1 = s', a_1= a'| s_0 = s , a_0 = a) \mathbb{E}\left[\sum_{t=1}^T \gamma^t r(s_t, a_t) | s_1=s', a_1 = a'\right] \\
& = r(s, a) + \sum_{s', a'} p(s_1 = s', a_1= a'| s_0 = s , a_0 = a) \mathbb{E}\left[\sum_{t=0}^{T-1} \gamma^{t+1} r(s_t, a_t) | s_1=s', a_1 = a'\right] \\
\end{align*}$$
 
and since $T=\infty$, 
$$ Q^\pi(s,a) = \mathbb{E}_{s', a' \sim p(.,.|s,a)} \left[ r(s,a) + \gamma Q^\pi(s',a') \right] $$



2)  
$$Q^*(s,a) = \max_\pi Q^\pi(s,a)$$

and, using the previous result, we kown that 
>$$\begin{align*}Q^\pi(s,a) &=  \mathbb{E}_{s', a' \sim p(.,.|s,a)}\left[ r(s,a) + \gamma Q^\pi(s',a')\right]\\
&= r(s,a) +  \mathbb{E}_{s' \sim p(.|s,a)}\left[\gamma Q^\pi(s', \pi(s'))\right]\\
&= r(s,a) + \gamma \sum_{s'} p(s_1 = s' | s_0 = s, a_0 = a)Q^\pi(s', \pi(s'))
\end{align*}$$

therefore  
$$\begin{align*}Q^*(s,a) &= \max_\pi \left\{ r(s,a) + \gamma \sum_{s'} p(s_1 = s' | s_0 = s, a_0 = a)Q^\pi(s', \pi(s')) \right\} \\
& = r(s,a) + \gamma \sum_{s'} p(s_1 = s' | s_0 = s, a_0 = a) \max_\pi Q^\pi(s', \pi(s')) \\
& = r(s,a) + \gamma \sum_{s'} p(s_1 = s' | s_0 = s, a_0 = a) Q^*(s', \pi^*(s'))  
\end{align*}$$
And since $\pi^*(s') = arg\max_{a'}Q^*(s', a')$ then 

>$$ \begin{align*} Q^*(s,a) & = r(s,a) + \gamma \sum_{s'} p(s_1 = s' | s_0 = s, a_0 = a) \max_{a'} Q^*(s', a')\\
& = \mathbb{E}_{s' \sim \pi^*(.|s,a)}\left[ r(s,a) + \gamma \max_{a'} Q^*(s', a')\right]
\end{align*}$$


3) The Deep Reinforcement Learning aims at finding the $\theta$ parameters that optimize the agent's behavior. However, here it is the Q-function that we want to optimize so we must define the objective function in relation to it. According to the previous question, the optimal Q-function is defined as  
$$ Q^*(s,a) = \mathbb{E}_{s' \sim \pi^*(.|s,a)}\left[ r(s,a) + \gamma \max_{a'} Q^*(s', a')\right]$$

and since the objective function is often defined by 

$$\mathcal{L}(\theta) = || Q^*(s,a) - Q(s,a,\theta) ||^2$$

then 

$$\begin{align*} \mathcal{L}(\theta) & = ||\mathbb{E}_{s' \sim \pi^*(.|s,a)}\left[ r(s,a) + \gamma \max_{a'} Q^*(s', a')\right]  - Q(s,a,\theta) ||^2 \\
& = \mathbb{E}_{s' \sim \pi^*(.|s,a)} \left[ || r(s,a) + \gamma \max_{a'} Q^*(s', a') - Q(s,a,\theta) ||^2 \right]
\end{align*}$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [14]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):

        if(len(self.memory)>=self.max_memory):
            random_remove_index = np.random.randint(0, len(self.memory), size=1)[0]
            self.memory.pop(random_remove_index)
        
        self.memory.append(m)
        return 

    def random_access(self):
        random_index = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [15]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [16]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size


    def learned_act(self, s):
        Q = self.model.predict(s.reshape(-1,5,5, self.n_state)).ravel()
        return np.argmax(Q)
                
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        #s_ = state, n_s = next_state, a_ = action, r_ = reward, game_over or not
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        input_next_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        rewards = np.zeros(self.batch_size)
        actions = np.zeros(self.batch_size)
        gameover = np.zeros(self.batch_size)
        
        for i in range(self.batch_size):
            input_states[i], input_next_states[i], actions[i], rewards[i], gameover[i] = self.memory.random_access()
    
        Q_present = self.model.predict(input_states)
        Q_next = self.model.predict(input_next_states)
        target_q = Q_present.copy()
        
        for j in range(self.batch_size):
            #state, next_state, action, reward, gameover = 
            if gameover[j]:
                target_q[j, int(actions[j])] = rewards[j]
            else:
                target_q[j, int(actions[j])] = rewards[j] + self.discount * np.max(Q_next[j])
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(4))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/300 | Loss 0.0293 | Win/lose count 6.0/4.0 (2.0)
Epoch 001/300 | Loss 0.0337 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/300 | Loss 0.0113 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/300 | Loss 0.0060 | Win/lose count 3.0/7.0 (-4.0)
Epoch 004/300 | Loss 0.0070 | Win/lose count 1.5/2.0 (-0.5)
Epoch 005/300 | Loss 0.0048 | Win/lose count 3.0/6.0 (-3.0)
Epoch 006/300 | Loss 0.0103 | Win/lose count 3.0/5.0 (-2.0)
Epoch 007/300 | Loss 0.0066 | Win/lose count 0.5/0 (0.5)
Epoch 008/300 | Loss 0.0027 | Win/lose count 2.0/2.0 (0.0)
Epoch 009/300 | Loss 0.0359 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/300 | Loss 0.0335 | Win/lose count 2.0/5.0 (-3.0)
Epoch 011/300 | Loss 0.0017 | Win/lose count 3.5/5.0 (-1.5)
Epoch 012/300 | Loss 0.0052 | Win/lose count 3.5/0 (3.5)
Epoch 013/300 | Loss 0.0247 | Win/lose count 5.5/1.0 (4.5)
Epoch 014/300 | Loss 0.0063 | Win/lose count 2.5/4.0 (-1.5)
Epoch 015/300 | Loss 0.0294 | Win/lose count 6.5/2.0 (4.5)
Epoch 016/300 | Loss 0.0020 | Win/lose count 5.0/3.

Epoch 139/300 | Loss 0.0053 | Win/lose count 7.0/2.0 (5.0)
Epoch 140/300 | Loss 0.0050 | Win/lose count 13.0/4.0 (9.0)
Epoch 141/300 | Loss 0.0073 | Win/lose count 3.0/0 (3.0)
Epoch 142/300 | Loss 0.0051 | Win/lose count 13.5/3.0 (10.5)
Epoch 143/300 | Loss 0.0084 | Win/lose count 4.0/0 (4.0)
Epoch 144/300 | Loss 0.0052 | Win/lose count 12.5/1.0 (11.5)
Epoch 145/300 | Loss 0.0046 | Win/lose count 16.0/2.0 (14.0)
Epoch 146/300 | Loss 0.0030 | Win/lose count 15.0/1.0 (14.0)
Epoch 147/300 | Loss 0.0035 | Win/lose count 12.0/3.0 (9.0)
Epoch 148/300 | Loss 0.0547 | Win/lose count 3.5/2.0 (1.5)
Epoch 149/300 | Loss 0.0029 | Win/lose count 10.5/6.0 (4.5)
Epoch 150/300 | Loss 0.0546 | Win/lose count 11.5/4.0 (7.5)
Epoch 151/300 | Loss 0.0023 | Win/lose count 9.0/1.0 (8.0)
Epoch 152/300 | Loss 0.0059 | Win/lose count 11.5/1.0 (10.5)
Epoch 153/300 | Loss 0.0045 | Win/lose count 17.0/2.0 (15.0)
Epoch 154/300 | Loss 0.0096 | Win/lose count 17.0/4.0 (13.0)
Epoch 155/300 | Loss 0.0509 | Win/lose cou

Epoch 276/300 | Loss 0.0045 | Win/lose count 14.5/3.0 (11.5)
Epoch 277/300 | Loss 0.0034 | Win/lose count 17.0/3.0 (14.0)
Epoch 278/300 | Loss 0.0021 | Win/lose count 12.0/5.0 (7.0)
Epoch 279/300 | Loss 0.0027 | Win/lose count 11.0/1.0 (10.0)
Epoch 280/300 | Loss 0.0502 | Win/lose count 20.5/3.0 (17.5)
Epoch 281/300 | Loss 0.0039 | Win/lose count 8.0/1.0 (7.0)
Epoch 282/300 | Loss 0.0043 | Win/lose count 9.0/1.0 (8.0)
Epoch 283/300 | Loss 0.0035 | Win/lose count 14.0/4.0 (10.0)
Epoch 284/300 | Loss 0.0493 | Win/lose count 8.5/1.0 (7.5)
Epoch 285/300 | Loss 0.0020 | Win/lose count 9.5/0 (9.5)
Epoch 286/300 | Loss 0.0024 | Win/lose count 23.0/0 (23.0)
Epoch 287/300 | Loss 0.0050 | Win/lose count 9.0/0 (9.0)
Epoch 288/300 | Loss 0.0032 | Win/lose count 3.0/0 (3.0)
Epoch 289/300 | Loss 0.0063 | Win/lose count 13.5/4.0 (9.5)
Epoch 290/300 | Loss 0.0024 | Win/lose count 16.5/3.0 (13.5)
Epoch 291/300 | Loss 0.0046 | Win/lose count 2.0/0 (2.0)
Epoch 292/300 | Loss 0.0047 | Win/lose count 14.5/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [20]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(16, (3,3), input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(8, (2,2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [21]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/300 | Loss 0.0047 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/300 | Loss 0.0050 | Win/lose count 1.0/5.0 (-4.0)
Epoch 002/300 | Loss 0.0113 | Win/lose count 5.5/5.0 (0.5)
Epoch 003/300 | Loss 0.0614 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/300 | Loss 0.0026 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/300 | Loss 0.0031 | Win/lose count 4.5/8.0 (-3.5)
Epoch 006/300 | Loss 0.0023 | Win/lose count 5.0/2.0 (3.0)
Epoch 007/300 | Loss 0.0060 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/300 | Loss 0.0059 | Win/lose count 6.0/1.0 (5.0)
Epoch 009/300 | Loss 0.0053 | Win/lose count 8.5/6.0 (2.5)
Epoch 010/300 | Loss 0.0048 | Win/lose count 9.0/6.0 (3.0)
Epoch 011/300 | Loss 0.0058 | Win/lose count 8.0/8.0 (0.0)
Epoch 012/300 | Loss 0.0066 | Win/lose count 10.0/5.0 (5.0)
Epoch 013/300 | Loss 0.0033 | Win/lose count 6.0/3.0 (3.0)
Epoch 014/300 | Loss 0.0039 | Win/lose count 10.5/1.0 (9.5)
Epoch 015/300 | Loss 0.0031 | Win/lose count 3.5/6.0 (-2.5)
Epoch 016/300 | Loss 0.0040 | Win/lose count 2.5/2.

Epoch 137/300 | Loss 0.0047 | Win/lose count 14.5/2.0 (12.5)
Epoch 138/300 | Loss 0.0021 | Win/lose count 15.0/2.0 (13.0)
Epoch 139/300 | Loss 0.0060 | Win/lose count 12.5/5.0 (7.5)
Epoch 140/300 | Loss 0.0029 | Win/lose count 15.5/3.0 (12.5)
Epoch 141/300 | Loss 0.0498 | Win/lose count 4.5/2.0 (2.5)
Epoch 142/300 | Loss 0.0029 | Win/lose count 16.0/3.0 (13.0)
Epoch 143/300 | Loss 0.0030 | Win/lose count 4.5/1.0 (3.5)
Epoch 144/300 | Loss 0.0015 | Win/lose count 10.0/3.0 (7.0)
Epoch 145/300 | Loss 0.0017 | Win/lose count 7.0/2.0 (5.0)
Epoch 146/300 | Loss 0.0022 | Win/lose count 15.5/2.0 (13.5)
Epoch 147/300 | Loss 0.0019 | Win/lose count 7.5/3.0 (4.5)
Epoch 148/300 | Loss 0.0015 | Win/lose count 2.5/0 (2.5)
Epoch 149/300 | Loss 0.0037 | Win/lose count 1.0/1.0 (0.0)
Epoch 150/300 | Loss 0.0036 | Win/lose count 12.5/5.0 (7.5)
Epoch 151/300 | Loss 0.0024 | Win/lose count 21.0/3.0 (18.0)
Epoch 152/300 | Loss 0.0021 | Win/lose count 12.5/3.0 (9.5)
Epoch 153/300 | Loss 0.0065 | Win/lose cou

Epoch 273/300 | Loss 0.0038 | Win/lose count 15.0/3.0 (12.0)
Epoch 274/300 | Loss 0.0022 | Win/lose count 13.0/2.0 (11.0)
Epoch 275/300 | Loss 0.0039 | Win/lose count 6.0/1.0 (5.0)
Epoch 276/300 | Loss 0.0041 | Win/lose count 20.5/3.0 (17.5)
Epoch 277/300 | Loss 0.0014 | Win/lose count 21.5/5.0 (16.5)
Epoch 278/300 | Loss 0.0013 | Win/lose count 13.5/3.0 (10.5)
Epoch 279/300 | Loss 0.0024 | Win/lose count 18.5/0 (18.5)
Epoch 280/300 | Loss 0.0013 | Win/lose count 19.0/2.0 (17.0)
Epoch 281/300 | Loss 0.0023 | Win/lose count 19.0/3.0 (16.0)
Epoch 282/300 | Loss 0.0033 | Win/lose count 13.5/3.0 (10.5)
Epoch 283/300 | Loss 0.0033 | Win/lose count 20.5/0 (20.5)
Epoch 284/300 | Loss 0.0035 | Win/lose count 15.5/3.0 (12.5)
Epoch 285/300 | Loss 0.0013 | Win/lose count 18.5/2.0 (16.5)
Epoch 286/300 | Loss 0.0029 | Win/lose count 16.5/1.0 (15.5)
Epoch 287/300 | Loss 0.0026 | Win/lose count 11.5/2.0 (9.5)
Epoch 288/300 | Loss 0.0050 | Win/lose count 26.5/2.0 (24.5)
Epoch 289/300 | Loss 0.0048 | W

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [22]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 5.0/0. Average score (3.5)
Win/lose count 3.0/0. Average score (3.3333333333333335)
Win/lose count 17.5/0. Average score (6.875)
Win/lose count 7.0/0. Average score (6.9)
Win/lose count 26.5/0. Average score (10.166666666666666)
Win/lose count 5.0/0. Average score (9.428571428571429)
Win/lose count 14.0/0. Average score (10.0)
Win/lose count 13.5/0. Average score (10.38888888888889)
Win/lose count 5.0/0. Average score (9.85)
Win/lose count 5.0/0. Average score (9.409090909090908)
Win/lose count 8.5/0. Average score (9.333333333333334)
Win/lose count 2.0/0. Average score (8.76923076923077)
Win/lose count 3.5/0. Average score (8.392857142857142)
Win/lose count 3.5/0. Average score (8.066666666666666)
Win/lose count 17.5/0. Average score (8.65625)
Win/lose count 5.0/0. Average score (8.441176470588236)
Win/lose count 7.5/0. Average score (8.38888888888889)
Win/lose count 4.5/0. Average score (8.18421052631579)
Win/lo

In [30]:
HTML(display_videos('cnn_test'+str(epochs_test-10)+'.mp4'))

In [33]:
HTML(display_videos('fc_test'+str(epochs_test-10)+'.mp4'))

According to these results, the DQN_CNN algorithm is better than the DQN_NN algorithm. Moreover, the temperature allows to increase or decrease the cheese in the grid. Indeed, if we increase the temperature then there is more cheese and if we decrease the temperature then there is less cheese in the grid.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [51]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        reward = 0
        if train:
            reward = - self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        # 3 "feature" states instead of 2
        
        self.t = self.t + 1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state
    

In [52]:
def train_explore(agent,env,epoch,prefix=''):
    
    score = 0
    loss = 0

    t = 0
    
    for e in range(epoch):
        
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        
        t +=1 
        agent.set_epsilon(agent.epsilon ** (t/20))
        
        game_over = False

        win = 0
        lose = 0
        
        
        while not game_over:
            
            #agent.set_epsilon(agent.epsilon ** (t/20))
            
            # The agent performs an actionFal
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

In [53]:
# Training
epochs_train = 500

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 1,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/500 | Loss 0.0098 | Win/lose count 8.0/23.800000000000054 (-15.800000000000054)
Epoch 001/500 | Loss 0.0030 | Win/lose count 9.0/24.600000000000087 (-15.600000000000087)
Epoch 002/500 | Loss 0.0005 | Win/lose count 10.400000000000004/28.60000000000005 (-18.200000000000045)
Epoch 003/500 | Loss 0.0019 | Win/lose count 9.200000000000001/33.6000000000001 (-24.400000000000098)
Epoch 004/500 | Loss 0.0004 | Win/lose count 6.800000000000002/28.200000000000074 (-21.400000000000073)
Epoch 005/500 | Loss 0.0038 | Win/lose count 6.100000000000001/30.300000000000125 (-24.200000000000124)
Epoch 006/500 | Loss 0.0113 | Win/lose count 9.600000000000005/38.00000000000012 (-28.400000000000116)
Epoch 007/500 | Loss 0.0002 | Win/lose count 12.100000000000005/42.90000000000014 (-30.800000000000136)
Epoch 008/500 | Loss 0.0038 | Win/lose count 6.800000000000002/34.400000000000105 (-27.600000000000104)
Epoch 009/500 | Loss 0.0041 | Win/lose count 8.000000000000002/30.000000000000117 (-22.00000000

Epoch 081/500 | Loss 0.0000 | Win/lose count 2.4/20.50000000000002 (-18.100000000000023)
Epoch 082/500 | Loss 0.0021 | Win/lose count 3.9999999999999996/21.10000000000003 (-17.10000000000003)
Epoch 083/500 | Loss 0.0011 | Win/lose count 3.5999999999999996/19.200000000000003 (-15.600000000000003)
Epoch 084/500 | Loss 0.0056 | Win/lose count 4.8/19.900000000000016 (-15.100000000000016)
Epoch 085/500 | Loss 0.0008 | Win/lose count 9.600000000000003/17.69999999999998 (-8.099999999999978)
Epoch 086/500 | Loss 0.0011 | Win/lose count 5.2/18.799999999999997 (-13.599999999999998)
Epoch 087/500 | Loss 0.0032 | Win/lose count 2.8/19.400000000000006 (-16.600000000000005)
Epoch 088/500 | Loss 0.0016 | Win/lose count 8.400000000000002/17.999999999999986 (-9.599999999999984)
Epoch 089/500 | Loss 0.0006 | Win/lose count 8.800000000000002/17.899999999999984 (-9.099999999999982)
Epoch 090/500 | Loss 0.0000 | Win/lose count 3.1999999999999997/20.300000000000022 (-17.100000000000023)
Epoch 091/500 | Loss

Epoch 163/500 | Loss 0.0002 | Win/lose count 12.800000000000006/19.9 (-7.0999999999999925)
Epoch 164/500 | Loss 0.0010 | Win/lose count 14.000000000000007/20.600000000000037 (-6.60000000000003)
Epoch 165/500 | Loss 0.0095 | Win/lose count 10.000000000000004/18.599999999999998 (-8.599999999999994)
Epoch 166/500 | Loss 0.0001 | Win/lose count 3.9999999999999996/19.1 (-15.100000000000001)
Epoch 167/500 | Loss 0.0100 | Win/lose count 10.800000000000004/19.39999999999999 (-8.599999999999987)
Epoch 168/500 | Loss 0.0033 | Win/lose count 0.8/19.900000000000013 (-19.100000000000012)
Epoch 169/500 | Loss 0.0011 | Win/lose count 14.000000000000007/18.599999999999994 (-4.599999999999987)
Epoch 170/500 | Loss 0.0003 | Win/lose count 15.200000000000008/16.29999999999996 (-1.0999999999999535)
Epoch 171/500 | Loss 0.0000 | Win/lose count 8.000000000000002/19.099999999999987 (-11.099999999999985)
Epoch 172/500 | Loss 0.0053 | Win/lose count 12.400000000000006/19.000000000000007 (-6.600000000000001)
Ep

Epoch 245/500 | Loss 0.0025 | Win/lose count 11.200000000000005/20.300000000000026 (-9.100000000000021)
Epoch 246/500 | Loss 0.0086 | Win/lose count 11.200000000000005/18.299999999999994 (-7.099999999999989)
Epoch 247/500 | Loss 0.0001 | Win/lose count 14.800000000000008/16.399999999999963 (-1.5999999999999552)
Epoch 248/500 | Loss 0.1569 | Win/lose count 1.6/19.70000000000001 (-18.10000000000001)
Epoch 249/500 | Loss 0.0002 | Win/lose count 9.600000000000003/17.69999999999998 (-8.099999999999978)
Epoch 250/500 | Loss 0.0339 | Win/lose count 6.000000000000001/18.599999999999994 (-12.599999999999994)
Epoch 251/500 | Loss 0.0229 | Win/lose count 7.200000000000002/19.300000000000004 (-12.100000000000001)
Epoch 252/500 | Loss 0.0029 | Win/lose count 6.000000000000001/18.599999999999994 (-12.599999999999994)
Epoch 253/500 | Loss 0.0283 | Win/lose count 6.800000000000002/18.39999999999999 (-11.59999999999999)
Epoch 254/500 | Loss 0.0076 | Win/lose count 15.200000000000008/17.299999999999976 

Epoch 327/500 | Loss 0.0042 | Win/lose count 14.400000000000007/16.499999999999964 (-2.099999999999957)
Epoch 328/500 | Loss 0.0001 | Win/lose count 7.200000000000002/19.300000000000004 (-12.100000000000001)
Epoch 329/500 | Loss 0.0000 | Win/lose count 9.200000000000003/17.799999999999983 (-8.59999999999998)
Epoch 330/500 | Loss 0.0039 | Win/lose count 13.600000000000007/17.699999999999985 (-4.099999999999978)
Epoch 331/500 | Loss 0.0000 | Win/lose count 10.400000000000004/18.499999999999993 (-8.099999999999989)
Epoch 332/500 | Loss 0.0098 | Win/lose count 3.9999999999999996/19.1 (-15.100000000000001)
Epoch 333/500 | Loss 0.0301 | Win/lose count 16.000000000000007/17.099999999999977 (-1.0999999999999694)
Epoch 334/500 | Loss 0.0056 | Win/lose count 8.400000000000002/20.000000000000014 (-11.600000000000012)
Epoch 335/500 | Loss 0.0007 | Win/lose count 15.200000000000008/17.29999999999998 (-2.0999999999999712)
Epoch 336/500 | Loss 0.0017 | Win/lose count 6.800000000000002/18.399999999999

Epoch 407/500 | Loss 0.0006 | Win/lose count 6.000000000000001/18.599999999999994 (-12.599999999999994)
Epoch 408/500 | Loss 0.0052 | Win/lose count 17.200000000000003/15.79999999999996 (1.400000000000043)
Epoch 409/500 | Loss 0.0025 | Win/lose count 13.600000000000007/17.69999999999998 (-4.099999999999975)
Epoch 410/500 | Loss 0.0047 | Win/lose count 15.200000000000008/18.299999999999997 (-3.099999999999989)
Epoch 411/500 | Loss 0.0020 | Win/lose count 8.000000000000002/18.099999999999987 (-10.099999999999985)
Epoch 412/500 | Loss 0.0007 | Win/lose count 8.800000000000002/18.9 (-10.099999999999996)
Epoch 413/500 | Loss 0.0003 | Win/lose count 6.400000000000001/19.499999999999993 (-13.09999999999999)
Epoch 414/500 | Loss 0.0051 | Win/lose count 8.400000000000002/17.999999999999986 (-9.599999999999984)
Epoch 415/500 | Loss 0.0002 | Win/lose count 9.600000000000003/18.699999999999996 (-9.099999999999993)
Epoch 416/500 | Loss 0.0118 | Win/lose count 16.400000000000006/19.000000000000004 (

Epoch 489/500 | Loss 0.0702 | Win/lose count 11.200000000000005/18.299999999999994 (-7.099999999999989)
Epoch 490/500 | Loss 0.0011 | Win/lose count 17.200000000000003/18.800000000000004 (-1.6000000000000014)
Epoch 491/500 | Loss 0.0000 | Win/lose count 9.200000000000003/17.799999999999983 (-8.59999999999998)
Epoch 492/500 | Loss 0.0101 | Win/lose count 10.400000000000004/18.499999999999996 (-8.099999999999993)
Epoch 493/500 | Loss 0.0076 | Win/lose count 13.200000000000006/18.799999999999997 (-5.599999999999991)
Epoch 494/500 | Loss 0.0418 | Win/lose count 10.400000000000004/17.49999999999998 (-7.099999999999975)
Epoch 495/500 | Loss 0.0002 | Win/lose count 6.800000000000002/18.39999999999999 (-11.59999999999999)
Epoch 496/500 | Loss 0.0006 | Win/lose count 8.000000000000002/20.100000000000016 (-12.100000000000014)
Epoch 497/500 | Loss 0.0006 | Win/lose count 14.800000000000008/18.4 (-3.5999999999999908)
Epoch 498/500 | Loss 0.0068 | Win/lose count 11.600000000000005/24.20000000000003

In [54]:
HTML(display_videos('cnn_train_explore480.mp4'))

In [55]:
# Evaluation
epochs_test = 100
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore90.mp4'))

Win/lose count 8.0/0. Average score (8.0)
Win/lose count 3.5/0. Average score (5.75)
Win/lose count 11.5/1.0. Average score (7.333333333333333)
Win/lose count 4.5/0. Average score (6.625)
Win/lose count 8.5/0. Average score (7.0)
Win/lose count 12.0/1.0. Average score (7.666666666666667)
Win/lose count 17.0/2.0. Average score (8.714285714285714)
Win/lose count 19.5/1.0. Average score (9.9375)
Win/lose count 3.0/0. Average score (9.166666666666666)
Win/lose count 13.0/0. Average score (9.55)
Win/lose count 15.5/0. Average score (10.090909090909092)
Win/lose count 3.5/0. Average score (9.541666666666666)
Win/lose count 3.5/0. Average score (9.076923076923077)
Win/lose count 15.0/0. Average score (9.5)
Win/lose count 21.5/1.0. Average score (10.233333333333333)
Win/lose count 15.0/1.0. Average score (10.46875)
Win/lose count 5.0/1.0. Average score (10.088235294117647)
Win/lose count 17.5/1.0. Average score (10.444444444444445)
Win/lose count 13.0/1.0. Average score (10.526315789473685)
Wi

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***